# Étape 3 : Entraînement et Test de la Cascade de Traduction

Ce notebook permet de lancer l'entraînement ASR (Speech-to-Text) sur CPU et de tester la chaîne de traduction finale (Mina ➔ Éwé ➔ Français).

In [ ]:
import sys
import importlib
from pathlib import Path
if str(Path.cwd().parent) not in sys.path:
    sys.path.append(str(Path.cwd().parent))

import pandas as pd
from src.config import settings
import src.models.train_whisper_cpu
importlib.reload(src.models.train_whisper_cpu)
from src.models.train_whisper_cpu import train_whisper_on_cpu
from src.pipeline.translate_cascade import TranslationCascade

## 1. Entraînement ASR (Whisper sur CPU)

 (> **Note :** L'entraînement sur CPU est lent. Nous utilisons un gel des couches (freeze) à 90% pour accélérer le processus. Les hyperparamètres sont configurés dans `src/config/settings.py`.

In [ ]:
# Chargement du dataset préparé à l'étape 2
dataset_path = settings.PROCESSED_DIR / "bible_asr_dataset.csv"

if dataset_path.exists():
    df = pd.read_csv(dataset_path)
    print(f"Dataset chargé : {len(df)} exemples.")
    
    # Lancement de l'initialisation de l'entraînement
    # Note: assurez-vous d'avoir assez de RAM
    model, processor = train_whisper_on_cpu(None) # Remplacez None par votre dataset HuggingFace
else:
    print("Erreur : Le dataset ASR n'a pas encore été généré. Veuillez exécuter le notebook 02 d'abord.")

## 2. Test de la Cascade de Traduction (Mina ➔ Éwé ➔ Français)

Ici, nous testons la capacité du système à traduire du texte Mina vers le Français en passant par l'Éwé comme langue pivot.

In [ ]:
cascade = TranslationCascade()

test_phrases = [
    "Egbé nyé gbe gba.", 
    "Mose I 1:1",
    "Yesu fɛ̃ lɔlɔ̃."
]

for phrase in test_phrases:
    print("\n---")
    result = cascade.translate_mina_to_french(phrase)
    print(f"MINA : {result['mina']}")
    print(f"ÉWÉ  : {result['ewe']}")
    print(f"FR   : {result['french']}")